In [ ]:
import os
import glob

import pandas as pd

from django.db.utils import DataError
from django.core.exceptions import FieldDoesNotExist

In [ ]:
data_file_name = "resource_ODEEG-archaeology"

In [ ]:
dm_file = os.path.join(settings.BASE_DIR, 'odeeg','data', 'vases.xlsx')
data_file = os.path.join(settings.BASE_DIR, 'odeeg','data', 'dumps', '{}.xlsx'.format(data_file_name))

In [ ]:
df_dm = pd.read_excel(dm_file)
df_data = pd.read_excel(data_file)

In [ ]:
df_data.keys()

In [ ]:
current_class = Object
df_keys = df_data.keys()
nr_cols = len(df_keys)
for i, row in df_data.iterrows():
    temp_item = None
    col_counter = 0
    no_match_yet = True
    while col_counter < nr_cols and no_match_yet:
        cur_attr = df_keys[col_counter]
        try:
            cur_attr_type = current_class._meta.get_field(df_keys[col_counter]).get_internal_type()
        except FieldDoesNotExist:
            cur_attr_type = None
        col_counter += 1
        if cur_attr_type is not None and "{}".format(cur_attr_type) == "CharField" and isinstance(row[cur_attr], str):
            temp_item, _ = current_class.objects.get_or_create(
                legacy_id=row[cur_attr]
            )
            no_match_yet = False
    if temp_item:
        col_counter = 0
        while col_counter < nr_cols:
            cur_attr = df_keys[col_counter]
            try:
                cur_attr_type = current_class._meta.get_field(df_keys[col_counter]).get_internal_type()
            except FieldDoesNotExist:
                cur_attr_type = None
            if cur_attr_type is not None:
                print(cur_attr_type)
                if "{}".format(cur_attr_type) == "CharField" and isinstance(row[cur_attr], str):
                    setattr(temp_item, cur_attr, row[cur_attr][:249])
                elif "{}".format(cur_attr_type) == "ForeignKey" and isinstance(row[cur_attr], str):
                    fk  = current_class._meta.get_field(df_keys[col_counter])
                    temp_rel_obj, _ = fk.related_model.objects.get_or_create(legacy_id=row[cur_attr])
                    setattr(temp_item, cur_attr, temp_rel_obj)
                elif "{}".format(cur_attr_type) == "ManyToManyField" and isinstance(row[cur_attr], str):
                    fk = current_class._meta.get_field(df_keys[col_counter])
                    for x in row[cur_attr].split(";"):
                        temp_rel_obj, _ = fk.related_model.objects.get_or_create(legacy_id=x.strip())
                        rel_obj.append(temp_rel_obj)
                        temp_item.save()
                elif "{}".format(cur_attr_type) == "TextField" and isinstance(row[cur_attr], str):
                    setattr(temp_item, cur_attr, row[cur_attr])
                elif "{}".format(cur_attr_type) == "DecimalField":
                    try:
                        value = float(row[cur_attr])
                    except ValueError:
                        print("{} can't be parsed as float".format(row[cur_attr]))
                        value = None
                    if value is not None:
                        setattr(temp_item, cur_attr, row[cur_attr])
            temp_item.save()
            col_counter += 1
        

In [ ]:
fk.related_model

In [ ]:
Shape.objects.all().delete()

In [ ]:
field = 